<a href="https://colab.research.google.com/github/tktmyd/ipynbs/blob/main/fdm/FDM5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2次元SH問題

In [ ]:
import sys
if 'google.colab' in sys.modules:
    print("Installing pygmt on Google Colab. It may take a few minutes.")
    ! pip install -q condacolab &> /dev/null
    import condacolab
    condacolab.install()
    ! mamba install pygmt &> /dev/null

In [ ]:
import numpy as np
import pygmt

ここでは，発展問題として，2次元SH問題のスタガードグリッド差分法数値シミュレーションを行います．3次元の運動方程式と構成関係式に対して，$x_2$ 成分の微分がゼロであるという仮定のもと，$x_1$-$x_3$断面における波動伝播を考えます．すると，運動方程式と構成関係式は2組の方程式に分離することができ，そのうちの片方は以下のようなものになります．
$$
\begin{align}
		 & \rho \frac{\partial v_2}{\partial t} =
		\frac{\partial \sigma_{13}}{\partial x} +
		\frac{\partial \sigma_{23}}{\partial z}
		\\
		 & \frac{\partial \sigma_{23}}{\partial t} =
		\mu
		\frac{\partial v_2}{\partial x_3}
    , \quad \frac{\partial \sigma_{12}}{\partial t} =
		\mu
		\frac{\partial v_2}{\partial x_1}
\end{align}
$$

もし$x_3$方向を深さとして取ると，$v_2$は面外方向に水平に振動する成分ですから，$SH$波を表します．ここに示していない残りの$v_1$, $v_3$に関する方程式は $P$-$SV$ 波を表します．

表記を簡単にするため，$x_1\rightarrow x$, $x_3 \rightarrow z$と置き換えます．すると上記の方程式は

$$
\begin{align}
		 & \frac{\partial v_y}{\partial t} =
		 \frac{1}{\rho} \left(
		\frac{\partial \sigma_{xy}}{\partial x} +
		\frac{\partial \sigma_{yz}}{\partial z}
		\right)
		\\
		 & \frac{\partial \sigma_{yz}}{\partial t} =
		\mu
		\frac{\partial v_y}{\partial z}
		%\\
		% &
    , \quad \frac{\partial \sigma_{xy}}{\partial t} =
		\mu
		\frac{\partial v_y}{\partial x}
\end{align}
$$

と表されます．もし$\mu$が空間一様ならば，運動方程式の時間微分に構成関係式を代入すると，

$$
\frac{\partial^2 v_y}{\partial t^2} = \frac{\mu}{\rho}
\left( \frac{\partial^2 v_y}{\partial x^2} + \frac{\partial^2 v_y}{\partial z^2} \right)
$$
となり，これは速度$\beta = \sqrt{\mu/\rho}$ の2次元波動方程式にほかなりません．

## 方程式の離散化

1次元の問題と同じように，まず空間微分から離散化しましょう．

$$
x_I = x_0 + I \Delta x, z_K = z_0 + K \Delta z
$$

というふうに，$x, z$方向をそれぞれ$\Delta x$, $\Delta z$で離散化します．そのときの値を，たとえば

$$
v_{y\,(I-1/2,K-1/2)}(t) = v_y (x_{I-1/2}, z_{K-1/2}, t)
$$
のように表します．スタガードグリッドでは必要に応じてグリッド位置が 1/2 だけズレるのも，1次元の問題と同じです．

うまく位置を組み合わせると，$SH$問題の方程式は，

$$
\begin{align}
\sigma_{xy\,(I, K-1/2)}(t+\Delta t/2)
& =
\sigma_{xy\,(I, K-1/2)}(t-\Delta t/2) +
\mu_{(I, K-1/2)}
\frac{v_{y,(I+1/2,K-1/2)} (t) - v_{y,(I-1/2,K-1/2)}(t)}{\Delta x} \Delta t
\\
 \sigma_{yz\,(I-1/2, K)}(t+\Delta t/2)
& = \sigma_{yz\,(I-1/2, K)}(t-\Delta t/2) +
	\mu_{(I-1/2, K)}
    \frac{v_{y,(I-1/2,K+1/2)}(t) - v_{y,(I-1/2,K-1/2)}(t)}{\Delta z} \Delta t
\\
v_{y\,(I-1/2, K-1/2)}(t+\Delta t)
		& =
		v_{y\,(I-1/2, K-1/2)}(t)
		\notag \\&  +
		\frac{1}{\rho_{(I-1/2, K-1/2)}} \left[
            \frac{\sigma_{xy\,(I, K-1/2)}(t+\Delta t/2) - \sigma_{xy\,(I-1, K-1/2)}(t+\Delta t/2)}{\Delta x}
            \right. \notag\\
            &\qquad\qquad\qquad \left.
            +
            \frac{\sigma_{yz\,(I-1/2, K)}(t+\Delta t/2) - \sigma_{yz\,(I-1/2, K-1)}(t+\Delta t/2)}{\Delta z}
        \right] \Delta t
\end{align}
$$

添字が多くて大変ですが，よく見ると，空間位置がすべて中心差分になっていること，左辺が右辺より未来で，過去の情報のアップデートのかたちになっていること，が確認できるでしょう．

表記を簡単にしてコンピュータ言語で実装しやすくするため，時間発展を右辺から左辺へのアップデート $\leftarrow$ 記号で表し，かつセル番号 $i$と$k$をそれぞれ

$$
(I-1) \Delta x < x  \le I \Delta x,
\quad
(K-1) \Delta z < z  \le K \Delta z,
$$

の範囲をあらわすものとして定義すると，上式は以下のように（まだしも）スッキリと書き表せます．

$$
\begin{align}
    &\sigma_{xy\,(i,k)} \leftarrow \sigma_{xy\,(i,k)} + \mu_{(i,k)} \frac{v_{y\,(i+1,k)} - v_{y\,(i,k)}}{\Delta x} \Delta t
    \\
    &\sigma_{yz\,(i,k)} \leftarrow \sigma_{yz\,(i,k)} + \mu_{(i,k)} \frac{v_{y\,(i,k+1)} - v_{y\,(i,k)}}{\Delta z} \Delta t
    \\
    &v_{y\,(i,k)} \leftarrow v_{y\,(i,k)} + \frac{1}{\rho_{(i,k)}} \left[ \frac{\sigma_{xy\,(i,k)} - \sigma_{xy\,(i-1,k)}}{\Delta x} + \frac{\sigma_{yz\,(i,k)} - \sigma_{yz\,(i,k-1)}}{\Delta z} \right] \Delta t
\end{align}
$$

## SHコードの実装

それでは，1次元コードと同じように実装していきます．まずはパラメタの設定とメモリの確保からです．

In [ ]:
nx = 800
nz = 400
nt = 1200
dx = 0.2  # km
dz = 0.2  # km
dt = 0.02 # seconds

rho = np.zeros([nx+2, nz+2]) # 密度
mu  = np.zeros([nx+2, nz+2]) # 合成率
Vy  = np.zeros([nx+2, nz+2]) # 粒子速度（SH波）
Sxy = np.zeros([nx+2, nz+2]) # 応力 σxy
Syz = np.zeros([nx+2, nz+2]) # 応力 σyz

続けて，構造モデルを設定します．$z=0$ を地表面として，$z>0$では固体，$z<0$では空気のかわりに真空とします．

In [ ]:
# 座標系
x = np.linspace(0, nx*dx, nx+1)
z = np.linspace(0, nz*dz, nz+1) - 5 # z=0 を地表面にするため，マイナスの値から始める

for k in range(nz+1):

    # 地表面より上
    # 密度は，ゼロで割れないので形式上小さな値を与える
    if z[k] < 0:
        mu [:,k] = 0.0 # 剛性率はゼロ
        rho[:,k] = 0.001
    else: # 地下
        vs = 3.0
        rho[:,k] = 2.7
        mu[:,k] = rho[:,k] * vs * vs

続けては初期条件です．地中のある一点とその周辺に，滑らかに変化する初期速度を与えます．

In [ ]:
W = 20
for k in range(nz//4-W, nz//4+W):
    for i in range(nx//2-W, nx//2+W):
        Vy[i,k] = np.cos(np.pi / 2.0 * (i - nx / 2 ) / W)**3 \
                * np.cos(np.pi / 2.0 * (k - nz / 4 ) / W)**3

n = 0 # time step

この状態を，PyGMTで可視化をしてみましょう．

In [ ]:
def plot_wavefield(x, z, Vy, tim):

    fig = pygmt.Figure()
    pygmt.makecpt(cmap='balance', series=[-0.1, 0.1], background=True)
    grd = pygmt.xyz2grd(region = [x[0], x[-1], z[0], z[-1]],
                        spacing = [dx, dz],
                        data = [ [x[i], z[k], Vy[i,k]] for i in range(nx+1) for k in range(nz+1)] )
    fig.grdimage(grid = grd, projection='X12c/-6c',
                region = [x[0], x[-1], z[0], z[-1]], cmap=True,
                frame = ['WS', 'xa40f20+lx [km]', 'ya30f15+lz [km]'])
    fig.plot(x=[x[0], x[-1]], y=[0, 0], pen='thinner,gray@30')
    fig.colorbar(frame = ['x+lVelocity amplitude'],position='x12.5c/0.2c+w5.5c/0.3c+e0.2c+v')
    fig.text(x=x[-1]*0.98, y=z[-1]*0.02, justify='RT',
             text=f"t = {tim:6.2f} s", font='8p,Helvetica,black')

    return fig

In [ ]:
plot_wavefield(x, z, Vy, 0.0)

`pygmt.xyz2grd` により等間隔のグリッドデータを作成し，`pygmt.makecpt` で作成したカラーマップに沿って可視化しています．詳しくはたとえば[このページ](https://tktmyd.github.io/pygmt-howto-jp/mesh.html)に解説があります．

$z=0$ に線が引かれているのは地表面で，それより上では$\mu=0$であり，かつ$\rho$は非常に小さな値をとります．「真空」という条件からは$\rho$もゼロにして良さそうですが，実際にはそうすると運動方程式の評価で密度の逆数が計算できなくなってしまいます．

それでは，メインの計算部分です．ここではあえて `nt` の値を小さな値に上書きして，短時間の計算を行います．また，コードブロックの先頭に `%time` と書くことで，そのブロックの実行時間を計測します．

In [ ]:
%%time

ntdec = 1
nt =50 # 最初の50ステップだけ計算

for n in range(nt+1):

    # 構成関係式
    for i in range(nx+1):
        for k in range(nz+1):

            dxV = (Vy[i+1,k] - Vy[i,k]) / dx
            dzV = (Vy[i,k+1] - Vy[i,k]) / dz

            Sxy[i,k] += mu[i,k] * dxV * dt
            Syz[i,k] += mu[i,k] * dzV * dt

    # 運動方程式
    for i in range(nx+1):
        for k in range(nz+1):
            dxSxy = (Sxy[i,k] - Sxy[i-1,k]) / dx
            dzSyz = (Syz[i,k] - Syz[i,k-1]) / dz

            Vy[i,k] += ( dxSxy + dzSyz ) / rho[i,k] * dt

    if n % ntdec == 0:
        print(f'\rcalculating time step ({n}/{nt}): max = {np.max(Vy):.3f}', end='')

print()

だいたい，1時間ステップあたり1秒弱の時間がかかると思います．実は，Pythonには **愚直にループを書くと計算がとても遅い** という顕著な欠点があります．将来のバージョンである程度の改善は見込まれているようですが，大規模な数値シミュレーションはそもそもPythonで書くことが困難です．大規模な数値シミュレーションは，FortranやC++といった（古くから使われている）プログラミング言語で書かれていることが多いです．

ですが，ここではあえてPythonで上記の計算を高速化してみます．以下のコードは上記とまったく同じことを行いますが，`i`, `k` 方向の `for` ループを排除しました．これだけで劇的に計算時間が変わります．

In [ ]:
%%time
ntdec = 1
nt =50 # 最初の50ステップだけ計算

# Pythonのループはとても遅いため，スライス演算をつかって x, z 方向のループを排除
for n in range(nt+1):

    # 構成関係式
    # 全グリッドの空間微分を一括で計算
    dxV = (Vy[1:nx+1,0:nz] - Vy[0:nx,0:nz]) / dx
    dzV = (Vy[0:nx,1:nz+1] - Vy[0:nx,0:nz]) / dz

    # 全グリッドの応力を一括で更新
    Sxy[0:nx,0:nz] += mu[0:nx,0:nz] * dxV * dt
    Syz[0:nx,0:nz] += mu[0:nx,0:nz] * dzV * dt

    # 運動方程式
    dxSxy = (Sxy[1:nx,1:nz] - Sxy[0:nx-1,1:nz]) / dx
    dzSyz = (Syz[1:nx,1:nz] - Syz[1:nx,0:nz-1]) / dz

    Vy[1:nx,1:nz] += ( dxSxy + dzSyz ) / rho[1:nx,1:nz] * dt

    if n % ntdec == 0:
        print(f'\rcalculating time step ({n}/{nt}): max = {np.max(Vy):.3f}', end='')

print()

だいたい100倍くらい計算が速くなりました．これはやや極端な例ですが，数値シミュレーションの世界では，同じ問題でもプログラムの書き方（アルゴリズム）によって，計算の効率が実現可能と不可能を分けるくらいに劇的に変わることがあります．そのため，計算コードの高速化，というのも実はとても大事な研究分野なのです．

それでは，後者のアルゴリズムを踏まえて，計算と可視化を一挙に行うコードを関数としてまとめてみます．
この関数は，地震波速度 `beta` の2次元リストを引数にとり，その媒質における地震波を計算します（ただし，与えられた `beta` をそのまま使うのではなく， `z<0` の部分は強制的に `0` にします）．その他のパラメタのいくつかも引数として変更できるようにしてあります．

In [ ]:
def gif_movie(figs, dpi=720, crop='0.5c'):

    """
    PyGMTのFigureオブジェクトのリストからGifアニメーションを作成する．Jupyter Notebook上で表示されるオブジェクトを返す．

    Parameters
    ----------
    figs : list of Figure
        PyGMTのFigureオブジェクトのリスト
    dpi : int, optional
        解像度 (default: 720)
    crop : str, optional
        余白のトリミング量 (default: '0.5c')

    Returns
    -------
    HTML : IPython.display.HTML
        Gifアニメーション
    """
    from IPython import display as dd
    import tempfile
    import base64
    import os

    with tempfile.TemporaryDirectory() as tmpdir:
        for i, fig in enumerate(figs):
            figname = f'plot_{i:05d}.png'
            print(f'\rsaving figs ... ({(i+1)/len(figs)*100:5.1f}%)', end='')
            fig.savefig(os.path.join(tmpdir, figname), dpi=dpi, crop=crop)
        print(' Done.')

        cmd1 = f'ffmpeg -i {tmpdir}/plot_%5d.png '
        cmd2 = f' -vf "scale=800:-1,split [a][b];[a] palettegen [p];[b][p] paletteuse" '
        cmd3 = f' {tmpdir}/out.gif > /dev/null 2>&1'
        print(f'making gif ... ', end='')
        os.system(cmd1 + cmd2 + cmd3)
        print(' Done.')

        with open(f'{tmpdir}/out.gif', 'rb') as f:
            b64 = base64.b64encode(f.read()).decode("ascii")

    return dd.HTML(f'<img src="data:image/gif;base64,{b64}" width="80%"/>')

In [ ]:
def fdmsh(beta, dx=0.2, dz=0.2, dt=0.02, nt=1200, W=20):

    nx = np.shape(beta)[0] - 2
    nz = np.shape(beta)[1] - 2


    rho = np.zeros([nx+2, nz+2]) # 密度
    mu  = np.zeros([nx+2, nz+2]) # 合成率
    Vy  = np.zeros([nx+2, nz+2]) # 粒子速度（SH波）
    Sxy = np.zeros([nx+2, nz+2]) # 応力 σxy
    Syz = np.zeros([nx+2, nz+2]) # 応力 σyz

    # 座標系
    x = np.linspace(0, nx*dx, nx+1)
    z = np.linspace(0, nz*dz, nz+1) - 5 # z=0 を地表面にするため，マイナスの値から始める

    for k in range(nz+1):

        # 地表面より上
        # 密度は，ゼロで割れないので形式上小さな値を与える
        if z[k] < 0:
            mu [:,k] = 0.0 # 剛性率はゼロ
            rho[:,k] = 0.001
            beta[:,k] = 0.0
        else: # 地下
            for i in range(nx+1):
                rho[i,k] = 2.7
                mu[i,k] = rho[i,k] * beta[i,k] * beta[i,k]

    W = 20
    for k in range(nz//4-W, nz//4+W):
        for i in range(nx//2-W, nx//2+W):
            Vy[i,k] = np.cos(np.pi / 2.0 * (i - nx / 2 ) / W)**3 \
                    * np.cos(np.pi / 2.0 * (k - nz / 4 ) / W)**3

    n = 0 # time step

    figs = []
    figs.append(plot_wavefield(x, z, Vy, n * dt))

    ntdec = 10

    for n in range(nt+1):

        # 構成関係式
        # 全グリッドの空間微分を一括で計算
        dxV = (Vy[1:nx+1,0:nz] - Vy[0:nx,0:nz]) / dx
        dzV = (Vy[0:nx,1:nz+1] - Vy[0:nx,0:nz]) / dz

        # 全グリッドの応力を一括で更新
        Sxy[0:nx,0:nz] += mu[0:nx,0:nz] * dxV * dt
        Syz[0:nx,0:nz] += mu[0:nx,0:nz] * dzV * dt

        # 運動方程式
        dxSxy = (Sxy[1:nx,1:nz] - Sxy[0:nx-1,1:nz]) / dx
        dzSyz = (Syz[1:nx,1:nz] - Syz[1:nx,0:nz-1]) / dz

        Vy[1:nx,1:nz] += ( dxSxy + dzSyz ) / rho[1:nx,1:nz] * dt

        if n % ntdec == 0:
            print(f'\rcalculating time step ({n}/{nt}): max = {np.max(Vy):.3f}', end='')
            figs.append(plot_wavefield(x, z, Vy, n * dt))

    print()

    return gif_movie(figs)


In [ ]:
fdmsh(np.ones([802, 402]) * 4.0)